#################################################
##      뉴스기사 키워드 추출 및 테마 추천       ##
#################################################

0. 라이브러리 임포트

In [131]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pickle
import re
import itertools

1. 뉴스기사 선언

In [132]:
article = """
인천 청라 전기차 화재 이후 중국산 배터리에 대한 안전 우려가 커지고 있는데요.

최근 6년간 전기차 화재를 분석했더니 국내산 배터리가 대다수를 차지했습니다.

물론 국산 배터리 점유율이 높기 때문인데, 화재에는 제조사 따질 것 없다는 지적이 나옵니다.

박효정 기자의 단독 보도입니다.

[기자]

지난 1일 차량 140여대를 태운 인천 청라 벤츠 전기차에는 중국 파라시스사의 배터리가 탑재됐고, 닷새 뒤 충남 금산에서 불이 난 기아EV6에는 SK온의 배터리가 사용됐습니다.

경기 용인 테슬라 화재의 경우 한중일 배터리를 혼용해 쓴다는 게 제조사 입장일 뿐, 정확히 밝혀지지 않았습니다.

민주당 전용기 의원실에 따르면 최근 6년간 발생한 전기차 화재 139건 가운데 배터리에서 불이 시작된 경우는 75건으로 절반을 넘었습니다.

배터리 제조사별로 분석했더니 LG엔솔이 43건으로 가장 많았고, SK온은 22건으로 두 번째를 차지했습니다.

일본 파나소닉 3건, 삼성SDI는 2건을 차지했고, 파라시스 등 중국 업체는 5건에 불과했습니다.

국내산 배터리 점유율이 62%를 넘는 만큼 화재 발생도 많은 것입니다.

하지만, 인천 청라 화재 이후 유독 중국산 배터리는 위험하다는 식의 우려가 퍼졌습니다.

<김필수 / 대림대학교 자동차학과 교수> "어디가 좋고 나쁘고에 대한 결론이 아무것도 없는데 막연하게 '중국은 나빠'라는 논리 심어주는 것은 굉장히 위험하고, 통상 문제를 일으키면 우리만 불리하다는 측면에서 지금의 포비아를 해결하는 데 도움이 안 되는 흐름이다."

화재 상황별로 보면 주차 중인 경우가 51건으로 가장 많아 인천 청라 화재 같은 대규모 피해가 충분히 발생할 수 있음을 경고했습니다.

<전용기 / 더불어민주당 의원> "리튬배터리 화재위험성이 지속적으로 증가하는 만큼 배터리 실명제를 도입해서 국민들의 안전을 도모해야 할 시기가 왔습니다."

정부는 모든 전기차 제조사에 배터리 정보 공개를 권고한 가운데, 다음 달 종합대책을 발표할 예정입니다.
"""

2. 텍스트 전처리

In [133]:
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

print(f'전처리 후 뉴스 기사 : {text}')

전처리 후 뉴스 기사 : 인천 청라 전기차 화재 이후 중국산 배터리 대한 안전 우려 최근 전기차 화재 분석 국내 산 배터리 대다수 차지 국산 배터리 점유 율 때문 화재 조사 것 지적 박효정 기자 단독 보도 기자 지난 차량 여대 태운 인천 청라 벤츠 전기차 중국 파라 시스 사의 배터리 탑재 닷새 뒤 충남 금산 불 난 기아 EV 6에는 SK 온 배터리 사용 경기 용인 테슬라 화재 경우 한중일 배터리 혼용 게 조사 입장 일 뿐 민주당 전용기 의원 실 최근 발생 전기차 화재 건 가운데 배터리 불 시작 경우 건 절반 배터리 사별 분석 LG 솔이 건 가장 SK 온 건 두 번 차지 일본 파나소닉 건 삼성 SDI 건 차지 파라 시스 등 중국 업체 건 국내 산 배터리 점유 율 를 만큼 화재 발생 것 인천 청라 화재 이후 유독 중국산 배터리 식 우려 김필수 대림 대학교 자동차 학과 교수 어디가 대한 결론 것 중국 논리 것 통상 문제 우리 측면 지금 포비아 해결 데 도움 안 흐름 화재 상황 별로 주차 중인 경우 건 가장 인천 청라 화재 대규모 피해 발생 수 음 경고 전용기 민주당 의원 리튬배터리 화재 위험성 지속 증가 만큼 배터리 실명 제 도입 국민 안전 도모 시기 정부 모든 전기차 조사 배터리 정보 공개 권고 가운데 다음 달 종합 대책 발표 예정


3. N-Gram 생성 및 SBERT

In [134]:
n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()
print(f'N-Gram 개수 : {len(n_gram_words)}')
print(f'N-Gram 다섯개 출력 : {n_gram_words[:5]}')

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #   Sentence Transformers
article_embedding = model.encode([article])
n_gram_embeddings = model.encode(n_gram_words)
print(f'SBERT 분석 후 뉴스 기사 : {article_embedding}')
print(f'SBERT 분석 후 N-Gram : {n_gram_embeddings}')

N-Gram 개수 : 269
N-Gram 다섯개 출력 : ['6에는' '6에는 sk' 'ev' 'ev 6에는' 'lg']
SBERT 분석 후 뉴스 기사 : [[ 4.09825206e-01 -1.68137729e-01 -2.95536846e-01 -1.32993385e-01
   4.90858406e-02 -9.32421565e-01  2.09249914e-01  1.13146767e-01
   5.12543499e-01  2.04607487e-01  3.91180992e-01 -5.67277431e-01
   1.15136378e-01 -4.53115046e-01 -8.21586132e-01  5.25742233e-01
   8.49156499e-01  5.13907313e-01 -2.35109672e-01 -4.35161412e-01
  -4.47654009e-01 -2.52219409e-01  7.37403750e-01  4.30733114e-01
   1.59103796e-01  5.52354515e-01  4.33113933e-01  3.95305991e-01
   3.45212787e-01  2.88467050e-01 -5.13190627e-01  5.98875523e-01
  -3.43194634e-01 -2.31633782e-01  1.24119036e-01  5.50748587e-01
  -1.02569795e+00  1.24906719e-01 -1.96893349e-01  1.82345435e-01
  -5.60152054e-01 -2.10818976e-01  2.94788092e-01  8.42746273e-02
   5.13804138e-01  5.14349937e-01  2.40292445e-01  3.05963606e-01
   4.91287589e-01 -5.21683358e-02  8.72647882e-01  2.45594203e-01
  -1.78920090e-01  1.05926365e-01 -5.16394615e-01  1.17

4. Max Sum Similarity

In [135]:
def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    #   뉴스 기사와 N-Gram의 유사도
    distances = cosine_similarity(article_embedding, n_gram_embeddings)
    #   N-Gram들 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)

    #   뉴스 기사와 유사도가 높은 N-Gram을 variety개 가져오기
    words_idx = list(distances.argsort()[0][-variety:])
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'뉴스 키워드 5개 : {news_keywords}')

뉴스 키워드 5개 : ['배터리 우려', '리튬배터리 화재', '최근 전기차', '유독 중국산', '전기차 중국']


5. 테마 가져오기

In [136]:
#       딕셔너리에서 key 값 가져오기
with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

keys = loaded_dict.keys()
keys_list = list(keys)

new_keys_list = [re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', key) for key in keys_list]     #   테마 전처리
key_embeddings = model.encode(new_keys_list)                                        #   테마  SBERT 분석

6. 코사인 거리 기반 & Max Sum Similarity 테마 추천

In [137]:
top_n = 5
distances = cosine_similarity(article_embedding, key_embeddings)
cosine_recommend = [keys_list[index] for index in distances.argsort()[0][-top_n:]]

# maxsum_recommend = max_sum_sim(article_embedding, key_embeddings, keys_list, top_n=3, variety=30)

article = article.replace('.', '.\n')
print(f'뉴스 기사 : \n{article}')
print(f'뉴스 키워드 5개 : {news_keywords}')

print(f'\n뉴스 기사와 연관 되어 있는 테마 5개')
for i in range(0,5):
    print(f'{i+1}. {cosine_recommend[-i]} : {loaded_dict[cosine_recommend[-i]]}')

# print(f'\n뉴스 기사와 연관될 수 있는 테마 3개')
# maxsum_index = 1
# for maxsum in maxsum_recommend:
#     print(f'{maxsum_index}. {maxsum} : {loaded_dict[maxsum]}')
#     maxsum_index += 1

뉴스 기사 : 
인천 청라 전기차 화재 이후 중국산 배터리에 대한 안전 우려가 커지고 있는데요.
최근 6년간 전기차 화재를 분석했더니 국내산 배터리가 대다수를 차지했습니다.
물론 국산 배터리 점유율이 높기 때문인데, 화재에는 제조사 따질 것 없다는 지적이 나옵니다.
박효정 기자의 단독 보도입니다.
[기자]지난 1일 차량 140여대를 태운 인천 청라 벤츠 전기차에는 중국 파라시스사의 배터리가 탑재됐고, 닷새 뒤 충남 금산에서 불이 난 기아EV6에는 SK온의 배터리가 사용됐습니다.
경기 용인 테슬라 화재의 경우 한중일 배터리를 혼용해 쓴다는 게 제조사 입장일 뿐, 정확히 밝혀지지 않았습니다.
민주당 전용기 의원실에 따르면 최근 6년간 발생한 전기차 화재 139건 가운데 배터리에서 불이 시작된 경우는 75건으로 절반을 넘었습니다.
배터리 제조사별로 분석했더니 LG엔솔이 43건으로 가장 많았고, SK온은 22건으로 두 번째를 차지했습니다.
일본 파나소닉 3건, 삼성SDI는 2건을 차지했고, 파라시스 등 중국 업체는 5건에 불과했습니다.
국내산 배터리 점유율이 62%를 넘는 만큼 화재 발생도 많은 것입니다.
하지만, 인천 청라 화재 이후 유독 중국산 배터리는 위험하다는 식의 우려가 퍼졌습니다.
<김필수 / 대림대학교 자동차학과 교수> "어디가 좋고 나쁘고에 대한 결론이 아무것도 없는데 막연하게 '중국은 나빠'라는 논리 심어주는 것은 굉장히 위험하고, 통상 문제를 일으키면 우리만 불리하다는 측면에서 지금의 포비아를 해결하는 데 도움이 안 되는 흐름이다.
"화재 상황별로 보면 주차 중인 경우가 51건으로 가장 많아 인천 청라 화재 같은 대규모 피해가 충분히 발생할 수 있음을 경고했습니다.
<전용기 / 더불어민주당 의원> "리튬배터리 화재위험성이 지속적으로 증가하는 만큼 배터리 실명제를 도입해서 국민들의 안전을 도모해야 할 시기가 왔습니다.
"정부는 모든 전기차 제조사에 배터리 정보 공개를 권고한 가운데, 다음 달 종합대책을 발표할 예정입니다.

뉴스 키워드 5개 : ['배